In [1]:
!pip install pyyaml requests


In [5]:
# Create the YAML content directly in Colab
yaml_content = '''
- headers:
    user-agent: fetch-synthetic-monitor
  method: GET
  name: fetch index page
  url: https://fetch.com/
- headers:
    user-agent: fetch-synthetic-monitor
  method: GET
  name: fetch careers page
  url: https://fetch.com/careers
- body: '{"foo":"bar"}'
  headers:
    content-type: application/json
    user-agent: fetch-synthetic-monitor
  method: POST
  name: fetch some fake post endpoint
  url: https://fetch.com/some/post/endpoint
- name: fetch rewards index page
  url: https://www.fetchrewards.com/
'''

# Save the YAML content to a file in Colab
with open('/content/config.yaml', 'w') as file:
    file.write(yaml_content)

# Verify that the file was created correctly
!cat /content/config.yaml



- headers:
    user-agent: fetch-synthetic-monitor
  method: GET
  name: fetch index page
  url: https://fetch.com/
- headers:
    user-agent: fetch-synthetic-monitor
  method: GET
  name: fetch careers page
  url: https://fetch.com/careers
- body: '{"foo":"bar"}'
  headers:
    content-type: application/json
    user-agent: fetch-synthetic-monitor
  method: POST
  name: fetch some fake post endpoint
  url: https://fetch.com/some/post/endpoint
- name: fetch rewards index page
  url: https://www.fetchrewards.com/


In [7]:
import yaml

# Define the load_config function
def load_config(file_path):
    with open(file_path, 'r') as file:
        endpoints = yaml.safe_load(file)
    return endpoints

# Load the configuration file
config_path = '/content/config.yaml'
endpoints = load_config(config_path)

# Display the loaded configuration to confirm
print(endpoints)



[{'headers': {'user-agent': 'fetch-synthetic-monitor'}, 'method': 'GET', 'name': 'fetch index page', 'url': 'https://fetch.com/'}, {'headers': {'user-agent': 'fetch-synthetic-monitor'}, 'method': 'GET', 'name': 'fetch careers page', 'url': 'https://fetch.com/careers'}, {'body': '{"foo":"bar"}', 'headers': {'content-type': 'application/json', 'user-agent': 'fetch-synthetic-monitor'}, 'method': 'POST', 'name': 'fetch some fake post endpoint', 'url': 'https://fetch.com/some/post/endpoint'}, {'name': 'fetch rewards index page', 'url': 'https://www.fetchrewards.com/'}]


In [8]:
import requests

def check_endpoint(endpoint):
    url = endpoint['url']
    method = endpoint.get('method', 'GET')
    headers = endpoint.get('headers', {})
    body = endpoint.get('body', None)

    try:
        if method == 'POST':
            response = requests.post(url, headers=headers, data=body, timeout=0.5)
        else:
            response = requests.get(url, headers=headers, timeout=0.5)

        # Check if response code is 2xx and latency is below 500 ms
        if response.status_code // 100 == 2 and response.elapsed.total_seconds() < 0.5:
            return 'UP'
        else:
            return 'DOWN'
    except (requests.RequestException, requests.Timeout):
        return 'DOWN'


In [9]:
import time

availability_tracker = {}

def monitor_endpoints(endpoints):
    while True:
        for endpoint in endpoints:
            domain = endpoint['url'].split('/')[2]  # Extract domain from URL
            status = check_endpoint(endpoint)

            # Update availability tracking
            if domain not in availability_tracker:
                availability_tracker[domain] = {'up': 0, 'total': 0}
            availability_tracker[domain]['total'] += 1
            if status == 'UP':
                availability_tracker[domain]['up'] += 1

            # Calculate and log availability percentage
            for domain, data in availability_tracker.items():
                availability_percentage = round(100 * data['up'] / data['total'])
                print(f"{domain} has {availability_percentage}% availability")

        time.sleep(15)  # Wait for 15 seconds before the next test cycle


In [10]:
monitor_endpoints(endpoints)


fetch.com has 100% availability
fetch.com has 100% availability
fetch.com has 67% availability
fetch.com has 67% availability
www.fetchrewards.com has 100% availability
fetch.com has 75% availability
www.fetchrewards.com has 100% availability
fetch.com has 80% availability
www.fetchrewards.com has 100% availability
fetch.com has 67% availability
www.fetchrewards.com has 100% availability
fetch.com has 67% availability
www.fetchrewards.com has 100% availability
fetch.com has 71% availability
www.fetchrewards.com has 100% availability
fetch.com has 75% availability
www.fetchrewards.com has 100% availability
fetch.com has 67% availability
www.fetchrewards.com has 100% availability
fetch.com has 67% availability
www.fetchrewards.com has 100% availability
fetch.com has 70% availability
www.fetchrewards.com has 100% availability
fetch.com has 73% availability
www.fetchrewards.com has 100% availability
fetch.com has 67% availability
www.fetchrewards.com has 100% availability
fetch.com has 67%

KeyboardInterrupt: 